In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Data Processing/Output')
cur_date = "081723"

library(readxl)
library(openxlsx)
library(tidyverse)
library(reshape2)
library(imputeLCMD)
library(vsn)

# reading in file
cytokine_df = data.frame(read_excel("Input/Cytokine_Data_050423.xlsx", sheet = 2))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some ma

Warning message:
“Expecting numeric in E1310 / R1310C5: got 'NA'”
Warning message:
“Expecting numeric in E1311 / R1311C5: got 'NA'”
Warning message:
“Expecting numeric in E1312 / R1312C5: got 'NA'”
Warning message:
“Expecting numeric in E1313 / R1313C5: got 'NA'”
Warning message:
“Expecting numeric in E1314 / R1314C5: got 'NA'”
Warning message:
“Expecting numeric in E1315 / R1315C5: got 'NA'”
Warning message:
“Expecting numeric in E1316 / R1316C5: got 'NA'”
Warning message:
“Expecting numeric in E1317 / R1317C5: got 'NA'”
Warning message:
“Expecting numeric in E1318 / R1318C5: got 'NA'”
Warning message:
“Expecting numeric in E1319 / R1319C5: got 'NA'”
Warning message:
“Expecting numeric in E1320 / R1320C5: got 'NA'”
Warning message:
“Expecting numeric in E1321 / R1321C5: got 'NA'”
Warning message:
“Expecting numeric in E1322 / R1322C5: got 'NA'”
Warning message:
“Expecting numeric in E1323 / R1323C5: got 'NA'”
Warning message:
“Expecting numeric in E1324 / R1324C5: got 'NA'”
Warning me

In [2]:
head(cytokine_df)

,Subject_No,Subject_ID,Condensate,Burn_Condition,Condensate_Conc,Cytokine,Cytokine_Conc,Cytokine_Conc_pslog2
,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
1,1,F_1,C,F,1,Eotaxin3,21.750958,4.507855
2,1,F_1,C,F,25,Eotaxin3,11.624224,3.658123
3,1,F_1,C,F,5,Eotaxin3,17.828714,4.234863
4,1,F_1,C,S,1,Eotaxin3,22.798534,4.572801
5,1,F_1,C,S,25,Eotaxin3,10.342567,3.503675
6,1,F_1,C,S,5,Eotaxin3,9.140652,3.342078


# Variance Stabilizing Transformation
This makes it so that the variance doesn't depend on the mean.
https://book.stat420.org/transformations.html

In [3]:
# first converting from long to wide format, which is necessary for the vsn function
wider_cytokine_df = cytokine_df %>%
# creating sample ids
    unite("SampleID", c(colnames(cytokine_df)[c(1,3:5)]), sep = "_") %>%
    select(-c("Cytokine_Conc_pslog2", "Subject_ID")) %>%
    pivot_wider(names_from = "SampleID", values_from = "Cytokine_Conc") %>%
    column_to_rownames(var = "Cytokine")

head(wider_cytokine_df)

,1_C_F_1,1_C_F_25,1_C_F_5,1_C_S_1,1_C_S_25,1_C_S_5,2_C_F_1,2_C_F_25,2_C_F_5,2_C_S_1,⋯,6_P_F_5,6_P_S_1,6_P_S_25,6_P_S_5,1_PBS_PBS_NA,2_PBS_PBS_NA,3_PBS_PBS_NA,4_PBS_PBS_NA,5_PBS_PBS_NA,6_PBS_PBS_NA
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Eotaxin3,21.7509578,11.6242239,17.8287139,22.79853427,10.3425666,9.1406516,9.4828925,13.0389850,19.1615054,12.6741020,⋯,0.01650485,7.193271,3.9063511,14.1320715,6.5164096,4.09746381,8.7016042,11.6088683,5.82224401,8.3443377
GMCSF,1.9105023,8.9311025,4.4253275,1.42736197,1.2257399,2.2074857,1.6934569,2.0050153,1.4620300,1.3916790,⋯,0.82353736,7.172952,1.3299958,1.6895796,1.2155607,4.82560648,2.7232247,0.4618191,0.56449034,0.7176056
IL10,0.3885503,0.5022661,0.4213744,0.05697711,0.2168990,0.1515419,0.2349819,0.3484975,0.2846984,0.3546597,⋯,4.61216618,3.019063,1.1693805,1.1973341,0.6951089,0.45756637,0.3243074,0.7354434,2.42846040,1.8449935
IL13,14.5879538,26.7203357,19.7468742,2.50176460,10.0623096,3.5578660,12.4418665,12.3599550,15.3831401,8.7577660,⋯,42.64164021,49.057905,49.0954161,38.3846026,32.3446737,51.55220801,22.4430018,40.1092582,45.00685705,48.1459764
IL15,0.3237601,0.2763980,0.2991661,0.30452873,0.2482430,0.3325163,0.3412690,0.3040695,0.2746376,0.4024510,⋯,0.36508633,1.998325,0.4682276,0.4886159,0.5040196,0.41471943,0.1600225,0.3335528,0.38695213,0.4394744
IL1a,1.1257898,7.2536320,1.2518342,0.37538110,0.3235307,0.2242163,1.7955959,2.6870800,0.3543066,0.4593748,⋯,0.86270506,1.527494,0.3319410,0.2703741,0.3943214,0.04431924,1.7533717,0.2403669,0.06152603,0.6930547


In [4]:
normalized_cytokine_matrix = justvsn(as.matrix(wider_cytokine_df), minDataPointsPerStratum = 17)
head(normalized_cytokine_matrix)

,1_C_F_1,1_C_F_25,1_C_F_5,1_C_S_1,1_C_S_25,1_C_S_5,2_C_F_1,2_C_F_25,2_C_F_5,2_C_S_1,⋯,6_P_F_5,6_P_S_1,6_P_S_25,6_P_S_5,1_PBS_PBS_NA,2_PBS_PBS_NA,3_PBS_PBS_NA,4_PBS_PBS_NA,5_PBS_PBS_NA,6_PBS_PBS_NA
Eotaxin3,5.4389243,3.6300246,4.6942720,6.1655224,3.70387083,4.5625947,3.9478316,4.3405976,5.1451499,4.5771705,⋯,-1.8134580,2.3074072,1.5619151,3.5366650,2.9308615,1.6264437,3.7579614,3.42968060,1.9332280,2.5966966
GMCSF,1.9654868,3.2596689,2.7091290,2.5101520,1.13701042,2.5775189,1.5252610,1.6757260,1.5138165,1.4002668,⋯,-0.3019734,2.3027130,0.3281911,0.7515293,0.5767490,1.8278784,2.1432472,-0.46152580,-0.3178089,-0.3937576
IL10,-0.1879661,-0.3128103,-0.3969353,0.5455062,0.00125879,-0.4494730,-0.9197442,-0.6590266,-0.5307119,-0.5384296,⋯,1.7171634,0.7196883,0.2012112,0.3686508,-0.1690997,-0.3673778,-0.3921523,-0.03985856,0.9042975,0.6253383
IL13,4.8643253,4.8121573,4.8408783,3.1729733,3.66652670,3.2340543,4.3363167,4.2638058,4.8299734,4.0444758,⋯,4.8152236,5.2509493,5.0321674,4.9515756,5.2294117,5.0548361,5.1073224,5.18866054,4.7173048,5.0724235
IL15,-0.4169083,-0.8104287,-0.7778766,1.1537427,0.05394121,0.2547010,-0.5152226,-0.8245002,-0.5697373,-0.3614577,⋯,-0.9665872,-0.2526983,-0.5394553,-0.4613801,-0.5811551,-0.4189149,-0.9660468,-0.71147529,-0.5135704,-0.8147901
IL1a,1.2291541,2.9693185,0.9693810,1.2892293,0.17314392,-0.1245073,1.6054883,2.0873572,-0.2862163,-0.1755552,⋯,-0.2571581,-1.0980020,-0.7409423,-0.8560486,-0.8845952,-0.9632026,1.5555735,-0.92467137,-0.9602393,-0.4263644


There's no missing data, so I'll convert the data back to a long format.

In [5]:
normalized_cytokine_df = data.frame(normalized_cytokine_matrix) %>%
    rownames_to_column(var = "Cytokine") %>%
    pivot_longer(cols = 2:79, names_to = "Sample_No", values_to = "Norm_Cytokine_Conc") %>%
    separate(Sample_No, into = c("Subject_No", "Condensate", "Burn_Condition", "Condensate_Conc"))

# dataframes automatically add X to col names if it starts with a number so removing that here
normalized_cytokine_df$Subject_No = as.numeric(gsub("X","", as.character(normalized_cytokine_df$Subject_No))) 
normalized_cytokine_df$Norm_Cytokine_Conc = as.numeric(normalized_cytokine_df$Norm_Cytokine_Conc)

# adding back in all other data
final_normalized_cytokine_df = full_join(normalized_cytokine_df, unique(cytokine_df[,1:2]))
final_normalized_cytokine_df = final_normalized_cytokine_df[,c(7,2,1,3:6)]

head(final_normalized_cytokine_df)

Joining with `by = join_by(Subject_No)`


Subject_ID,Subject_No,Cytokine,Condensate,Burn_Condition,Condensate_Conc,Norm_Cytokine_Conc
<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
F_1,1,Eotaxin3,C,F,1,5.438924
F_1,1,Eotaxin3,C,F,25,3.630025
F_1,1,Eotaxin3,C,F,5,4.694272
F_1,1,Eotaxin3,C,S,1,6.165522
F_1,1,Eotaxin3,C,S,25,3.703871
F_1,1,Eotaxin3,C,S,5,4.562595


In [6]:
# exporting
write.xlsx(final_normalized_cytokine_df, paste0(Output,"/", "Processed_Cyotkine_Data_", cur_date, ".xlsx"), 
           rowNames = FALSE)